# Python libraries

In [1]:
import ee
import geemap
import os
import numpy as np
import matplotlib.pyplot as plt

# Display map
To display the map the Map of geemap is used https://github.com/giswqs/geemap, which inherits from ipyleaft.Map https://ipyleaflet.readthedocs.io/en/latest/api_reference/map.html.

In [5]:
Map = geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

# Load Landsat 8 image
To load a the image from Landsat 8, a image from Google Earth Engine Data Catalog is filtered https://developers.google.com/earth-engine/datasets/catalog/landsat-8/ and displayed on the interactive map.

In [6]:
point = ee.Geometry.Point(-3.83502, 40.40595);
start = ee.Date('2019-07-01');
finish = ee.Date('2019-07-31');

# Filter a image from the landsat raw images collection, using bounds, date, and selecting the image with the least cloud cover
img = ee.ImageCollection('LANDSAT/LC08/C01/T1') \
            .filterBounds(point) \
            .filterDate(start, finish) \
            .sort('CLOUD_COVER', True) \
            .first()

# Select the corresponding bands for the multiespectral and panchromatic image
img_multi = img.select(['B4', 'B3', 'B2'])
img_pan = img.select(['B8'])

# Display the images on the map
Map.addLayer(img_multi)
Map.addLayer(img_pan)

# Center the map
Map.setCenter(-3.83502, 40.40595, 8)

# A. Select a region of interest (roi) using the map tools
Using the tools row placed on the left of the map, draw a rectangle inside of the image. This rectangle will be the region of interest

In [7]:
# feature to extract
feature = Map.draw_last_feature
roi = feature.geometry()

print('Polygon coordinates >>', roi.getInfo().get('coordinates'))

AttributeError: 'NoneType' object has no attribute 'geometry'

# B. Select a roi using the coordinates
Instead of drawing the rectangle, you can select an area by drawing a polygon using the GeoJson convention https://tools.ietf.org/html/rfc7946#section-3.1.6. There is a polygon drawn by default.

In [8]:
# feature to extract
roi = ee.Geometry.Polygon([
  [[-4.547875, 39.766127], [-4.547875, 40.551179], [-2.960148, 40.551179], [-2.960148, 39.766127], [-4.547875, 39.766127]]
]);

# add the polygon to the map
Map.addLayer(roi, {}, 'feature');

print('Polygon coordinates >>', roi.getInfo().get('coordinates'))

Polygon coordinates >> [[[-4.547875, 39.766127], [-2.960148, 39.766127], [-2.960148, 40.551179], [-4.547875, 40.551179], [-4.547875, 39.766127]]]


# Download the roi as a TIFF file
The next block generates the path to store the TIFF.

In [13]:
out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
filename_xs = os.path.join(out_dir, 'landsat_multi_2.tif')
filename_pan = os.path.join(out_dir, 'landsat_pan_2.tif')

### Download as raw images without resizing

* A TIFF file including all bands

In [14]:
geemap.ee_export_image(img_multi, filename=filename_xs, region=roi, file_per_band=False)
geemap.ee_export_image(img_pan, filename=filename_pan, region=roi, file_per_band=False)

Generating URL ...
Please wait ...
[WinError 32] El proceso no tiene acceso al archivo porque está siendo utilizado por otro proceso: 'C:\\Users\\Edwin Hurtado\\Downloads\\landsat_multi_2.zip'
Generating URL ...
Please wait ...
[WinError 32] El proceso no tiene acceso al archivo porque está siendo utilizado por otro proceso: 'C:\\Users\\Edwin Hurtado\\Downloads\\landsat_pan_2.zip'


* A TIFF file for each band

In [ ]:
geemap.ee_export_image(img_multi, filename=filename_xs, region=roi, file_per_band=True)
geemap.ee_export_image(img_pan, filename=filename_pan, region=roi, file_per_band=False)

### Download resized images

* A TIFF file including all bands

In [ ]:
# Set the image scale meters, 150 means 15 meters of resolution, change to the convenient value
img_scale = 150
geemap.ee_export_image(img_multi, filename=filename_xs, scale = img_scale, region=roi, file_per_band=False)
geemap.ee_export_image(img_pan, filename=filename_pan, scale = img_scale, region=roi, file_per_band=False)

* A TIFF file for each band (This option is recommended to proceed with the pansharpening module)

In [9]:
# Set the image scale meters, 150 means 15 meters of resolution, change to the convenient value
img_scale = 150
geemap.ee_export_image(img_multi, filename=filename_xs, scale = img_scale, region=roi, file_per_band=True)
geemap.ee_export_image(img_pan, filename=filename_pan, scale = img_scale, region=roi, file_per_band=False)

Generating URL ...
Please wait ...
[WinError 32] El proceso no tiene acceso al archivo porque está siendo utilizado por otro proceso: 'C:\\Users\\Jiahao\\Downloads\\landsat_multi.zip'
Generating URL ...
Please wait ...
[WinError 32] El proceso no tiene acceso al archivo porque está siendo utilizado por otro proceso: 'C:\\Users\\Jiahao\\Downloads\\landsat_pan.zip'
